## kinect_cut_timestamp

In [2]:
def filter_column(column, window_size):
    return column.rolling(window=window_size).mean()


In [3]:
#导入所用包
import pandas as pd
import numpy as np
import os
from datetime import datetime


def kinect_preprocess(folder_path):
    # 获取指定文件夹下的所有文件和文件夹
    contents = os.listdir(folder_path)
    timestamp_path="C:\\Users\\lyy\\Desktop\\Data\\Raw\\Timestamp"
    # 遍历01、02、。。。文件夹
    for item in contents[:]:
        item_path = os.path.join(folder_path, item)
        
        #浏览对应item的timestamp文件夹
        item_timestamp= os.path.join(timestamp_path,item)
        print('folder：' + item_timestamp)
        
        # 检查是否为文件夹
        if os.path.isdir(item_path):
            print('folder：' + item_path)
        
        #遍历item_kinect下的子文件first、second;遍历item_timestamp下的子文件first、second
        item_contents = os.listdir(item_path)#first和second文件名
        
        for t in item_contents[:]:
            item_path_time = os.path.join(item_path, t) #第item次子文件地址
            item_timestamp_time= os.path.join(item_timestamp, t) #第item次时间戳
            print('folder：'+item_path_time)
            print('folder：'+item_timestamp_time)
            
            #浏览item_timestamp的对应次数时间戳
            timestamp=pd.read_csv(item_timestamp_time+'.csv')
            
            
            file_contents = os.listdir(item_path_time)
            
            
            for file in file_contents[:4]:
                if file[:-4] == 'task_normal':
                    row = 0
                if file[:-4] == 'task_cup':
                    row = 1
                if file[:-4] == 'task_text':
                    row = 2
                if file[:-4] == 'task_web':
                    row = 3

                file_path = os.path.join(item_path_time, file)
                print('file：'+file_path)
                
                #将Kinect时间转化为时间戳
                ki=pd.read_csv(file_path)
                ki1=ki.copy()
                
                ki1['timestamp2']=0
                import time
                for i in range(0,len(ki1)):
                    time_num = ki1.iloc[i,0][:-8]
                    time_array = time.strptime(time_num, "%Y-%m-%d %H:%M:%S")
                    time_stamp = time.mktime(time_array)
                    time_stamp =time_stamp+float(ki1.iloc[i,0][-8:-4])
                    ki1.iloc[i,26]=time_stamp
    
                #时间戳截取
                start_kinect=timestamp.iloc[row,2][:-7]
                start_turn=timestamp.iloc[row,3][:-7]
                end_turn=timestamp.iloc[row,4][:-7]
                end_kinect=timestamp.iloc[row,5][:-7]
                
                #转化时间戳
                import time
                time_array_start_kinect = time.strptime(start_kinect, "%Y-%m-%d-%H:%M:%S")
                time_array_start_turn = time.strptime(start_turn, "%Y-%m-%d-%H:%M:%S")
                time_array_end_turn = time.strptime(end_turn, "%Y-%m-%d-%H:%M:%S")
                time_array_end_kinect = time.strptime(end_kinect, "%Y-%m-%d-%H:%M:%S")
                
                timestamp_start_kinect = time.mktime(time_array_start_kinect)
                timestamp_start_turn = time.mktime(time_array_start_turn)
                timestamp_end_turn = time.mktime(time_array_end_turn)
                timestamp_end_kinect = time.mktime(time_array_end_kinect)
                
                timestamp_start_kinect= timestamp_start_kinect+float(timestamp.iloc[row,2][-7:-3])
                timestamp_start_turn= timestamp_start_turn+float(timestamp.iloc[row,3][-7:-3])
                timestamp_end_turn= timestamp_end_turn+float(timestamp.iloc[row,4][-7:-3])
                timestamp_end_kinect= timestamp_end_kinect+float(timestamp.iloc[row,5][-7:-3])

                #截取Kinect能够捕捉到的开始结束数据
                #go:从start of kinect开始到start of turn结束
                ki_go=ki1.copy()
                for i in range(0,len(ki_go)):
                    timestamp2 = ki_go.iloc[i,26]
                    if timestamp2 >= timestamp_start_kinect :
                        ki_go.drop(ki_go.head(i).index,inplace=True)
                        break
                ki_go.reset_index(drop=True,inplace=True)
                for i in range(0,len(ki_go)):
                    timestamp2 = ki_go.iloc[i,26]
                    if timestamp2 > timestamp_start_turn :
                        ki_go.drop(ki_go.tail(len(ki_go)-i).index,inplace=True)
                        break
                ki_go=ki_go.drop(columns=['timestamp'])
                ki_go= ki_go[ki_go.columns[-1:].tolist() + ki_go.columns[:-1].tolist()]
                ki_go['timestamp2'] = ki_go['timestamp2'].astype(str)
                
                #将XYZ切分开
                ki_go2=ki_go.copy()
                ki_go2 = ki_go2[ki_go2['keypoint_0'] != 'None_tracked']
                ki_go2.reset_index(drop=True,inplace=True)
                for j in range(1,26):
                    ti=j-1  
                    ki_go2[['X'+str(ti),'Y'+str(ti),'Z'+str(ti)]]=ki_go2.iloc[:,j].str.split(expand=True)
                ki_go2=ki_go2.drop(ki_go2.columns[1:26], axis=1)
                #删除0所在行
                zero_rows_go = (ki_go2 == '0.0').any(axis=1)
                ki_go2 = ki_go2[~zero_rows_go]
                #进行平滑处理
                # 设定窗口大小
                window_size = 3
                # 对数据框的每一列应用滤波函数
                ki_go2.iloc[:,1:].apply(lambda x: filter_column(x, window_size))
                
                ki_go2.to_csv("C:\\Users\\lyy\\Desktop\\Data\\Processed\\Kinect_2m\\"+t+'\\go\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False)
                
              
                #turn:从start of turn 到 end of turn
                ki_turn=ki1.copy()
                for i in range(0,len(ki_turn)):
                    timestamp2 = ki_turn.iloc[i,26]
                    if timestamp2 >= timestamp_start_turn :
                        ki_turn.drop(ki_turn.head(i).index,inplace=True)
                        break
                ki_turn.reset_index(drop=True,inplace=True)
                for i in range(0,len(ki_turn)):
                    timestamp2 = ki_turn.iloc[i,26]
                    if timestamp2 > timestamp_end_turn :
                        ki_turn.drop(ki_turn.tail(len(ki_turn)-i).index,inplace=True)
                        break
                ki_turn=ki_turn.drop(columns=['timestamp'])
                ki_turn= ki_turn[ki_turn.columns[-1:].tolist() + ki_turn.columns[:-1].tolist()]
                ki_turn['timestamp2'] =ki_turn['timestamp2'].astype(str)
                #将XYZ切分开
                ki_turn2=ki_turn.copy()
                ki_turn2 = ki_turn2[ki_turn2['keypoint_0'] != 'None_tracked']
                ki_turn2.reset_index(drop=True,inplace=True)
                for j in range(1,26):
                    ti=j-1  
                    ki_turn2[['X'+str(ti),'Y'+str(ti),'Z'+str(ti)]]=ki_turn2.iloc[:,j].str.split(expand=True)
                ki_turn2=ki_turn2.drop(ki_turn2.columns[1:26], axis=1)  
                #删除0所在行
                zero_rows_turn =(ki_turn2 == '0.0').any(axis=1)
                ki_turn2 = ki_turn2[~zero_rows_turn]
                # 设定窗口大小
                window_size = 3
                # 对数据框的每一列应用滤波函数
                ki_turn2.iloc[:,1:].apply(lambda x: filter_column(x, window_size))
                ki_turn2.to_csv("C:\\Users\\lyy\\Desktop\\Data\\Processed\\Kinect_2m\\"+t+'\\turn\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False)
                
                #back:从end of turn 到end of kinect
                ki_back=ki1.copy()
                for i in range(0,len(ki_back)):
                    timestamp2 = ki_back.iloc[i,26]
                    if timestamp2 >= timestamp_end_turn :
                        ki_back.drop(ki_back.head(i).index,inplace=True)
                        break
                ki_back.reset_index(drop=True,inplace=True)
                for i in range(0,len(ki_back)):
                    timestamp2 = ki_back.iloc[i,26]
                    if timestamp2 > timestamp_end_kinect :
                        ki_back.drop(ki_back.tail(len(ki_back)-i).index,inplace=True)
                        break
                ki_back=ki_back.drop(columns=['timestamp'])
                ki_back= ki_back[ki_back.columns[-1:].tolist() + ki_back.columns[:-1].tolist()]
                ki_back['timestamp2'] =ki_back['timestamp2'].astype(str)
                #将XYZ切分开
                ki_back2=ki_back.copy()
                ki_back2 = ki_back2[ki_back2['keypoint_0'] != 'None_tracked']
                ki_back2.reset_index(drop=True,inplace=True)
                for j in range(1,26):
                    ti=j-1  
                    ki_back2[['X'+str(ti),'Y'+str(ti),'Z'+str(ti)]]=ki_back2.iloc[:,j].str.split(expand=True)
                ki_back2=ki_back2.drop(ki_back2.columns[1:26], axis=1)      
                #删除0所在行
                zero_rows_back =(ki_back2 == '0.0').any(axis=1)
                ki_back2 = ki_back2[~zero_rows_back]
                
                #交换左右点的坐标
                cols = list(ki_back2.columns)
                cols[13], cols[14], cols[15], cols[16], cols[17], cols[18],cols[19],cols[20],cols[21],cols[22],cols[23],cols[24],cols[25],cols[26],cols[27],cols[28],cols[29],cols[30],cols[31],cols[32],cols[33],cols[34],cols[35],cols[36],cols[37],cols[38],cols[39],cols[40],cols[41],cols[42],cols[43],cols[44],cols[45],cols[46],cols[47],cols[48],cols[49],cols[50],cols[51],cols[52],cols[53],cols[54],cols[55],cols[56],cols[57],cols[58],cols[59],cols[60],cols[64],cols[65],cols[66],cols[67],cols[68],cols[69],cols[70],cols[71],cols[72],cols[73],cols[74],cols[75] = cols[25],cols[26],cols[27],cols[28],cols[29],cols[30],cols[31],cols[32],cols[33],cols[34],cols[35],cols[36],cols[13], cols[14], cols[15], cols[16], cols[17], cols[18],cols[19],cols[20],cols[21],cols[22],cols[23],cols[24],cols[49],cols[50],cols[51],cols[52],cols[53],cols[54],cols[55],cols[56],cols[57],cols[58],cols[59],cols[60],cols[37],cols[38],cols[39],cols[40],cols[41],cols[42],cols[43],cols[44],cols[45],cols[46],cols[47],cols[48],cols[70],cols[71],cols[72],cols[73],cols[74],cols[75],cols[64],cols[65],cols[66],cols[67],cols[68],cols[69]
                ki_back2=ki_back2[cols]
                
                ki_back2.columns = ['timestamp2','X0','Y0','Z0','X1',	'Y1',	'Z1',	'X2',	'Y2',	'Z2',	'X3',	'Y3',	'Z3',	'X4',	'Y4',	'Z4',	'X5',	'Y5',	'Z5',	'X6',	'Y6',	'Z6',	'X7'	,'Y7',	'Z7',	'X8',	'Y8',	'Z8',	'X9'	,'Y9',	'Z9',	'X10',	'Y10',	'Z10',	'X11',	'Y11'	,'Z11'	,'X12'	,'Y12',	'Z12',	'X13',	'Y13',	'Z13',	'X14',	'Y14',	'Z14',	'X15'	,'Y15',	'Z15',	'X16',	'Y16',	'Z16',	'X17'	,'Y17',	'Z17'	,'X18'	,'Y18'	,'Z18',	'X19',	'Y19',	'Z19',	'X20',	'Y20',	'Z20',	'X21',	'Y21',	'Z21',	'X22',	'Y22','Z22',	'X23',	'Y23',	'Z23',	'X24',	'Y24',	'Z24']
                              
               # 设定窗口大小
                window_size = 3
                # 对数据框的每一列应用滤波函数
                ki_back2.iloc[:,1:].apply(lambda x: filter_column(x, window_size))
                ki_back2.to_csv("C:\\Users\\lyy\\Desktop\\Data\\Processed\\Kinect_2m\\"+t+'\\back\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False)

In [4]:
kinect_preprocess("C:\\Users\\lyy\\Desktop\\Data\\Raw\\Kinect")

folder：C:\Users\lyy\Desktop\Data\Raw\Timestamp\sample01
folder：C:\Users\lyy\Desktop\Data\Raw\Kinect\sample01
folder：C:\Users\lyy\Desktop\Data\Raw\Kinect\sample01\time01
folder：C:\Users\lyy\Desktop\Data\Raw\Timestamp\sample01\time01
file：C:\Users\lyy\Desktop\Data\Raw\Kinect\sample01\time01\task_cup.csv
file：C:\Users\lyy\Desktop\Data\Raw\Kinect\sample01\time01\task_normal.csv
file：C:\Users\lyy\Desktop\Data\Raw\Kinect\sample01\time01\task_text.csv
file：C:\Users\lyy\Desktop\Data\Raw\Kinect\sample01\time01\task_web.csv
folder：C:\Users\lyy\Desktop\Data\Raw\Kinect\sample01\time02
folder：C:\Users\lyy\Desktop\Data\Raw\Timestamp\sample01\time02
file：C:\Users\lyy\Desktop\Data\Raw\Kinect\sample01\time02\task_cup.csv
file：C:\Users\lyy\Desktop\Data\Raw\Kinect\sample01\time02\task_normal.csv
file：C:\Users\lyy\Desktop\Data\Raw\Kinect\sample01\time02\task_text.csv
file：C:\Users\lyy\Desktop\Data\Raw\Kinect\sample01\time02\task_web.csv
folder：C:\Users\lyy\Desktop\Data\Raw\Timestamp\sample02
folder：C:\Use